# Metrics of pre trained Mini-Replica and Clus-Replica ResNeXt-101 with augmented images

In [1]:
import pandas as pd
import sys
import pickle

sys.path.insert(0, "./../../../src/utils/")

from utils import get_train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from tqdm import tqdm

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec

from utils import get_scores

In [2]:
def get_averaged_df(score_dataframe):
    a = score_dataframe.groupby("model").agg(['mean', 'std'])
    a.columns = list(map('_'.join, a.columns.values))
    a["MP"] = round(a["MP_mean"], 2).astype(str) + " ± " + round(a["MP_std"], 2).astype(str)
    a["MMP"] = round(a["MMP_mean"], 2).astype(str) + " ± " + round(a["MMP_std"], 2).astype(str)
    a["mean_median_position"] = round(a["mean_median_position_mean"], 2).astype(str) + " ± " + round(a["mean_median_position_std"], 2).astype(str)
    a["MAP"] = round(a["MAP_mean"], 2).astype(str) + " ± " + round(a["MAP_std"], 2).astype(str)
    a["R@400"] = round(a["R@400_mean"], 2).astype(str) + " ± " + round(a["R@400_std"], 2).astype(str)
    a["R@200"] = round(a["R@200_mean"], 2).astype(str) + " ± " + round(a["R@200_std"], 2).astype(str)
    a["R@100"] = round(a["R@100_mean"], 2).astype(str) + " ± " + round(a["R@100_std"], 2).astype(str)
    a["R@50"] = round(a["R@50_mean"], 2).astype(str) + " ± " + round(a["R@50_std"], 2).astype(str)
    a["R@20"] = round(a["R@20_mean"], 2).astype(str) + " ± " + round(a["R@20_std"], 2).astype(str)

    b = a[["MP", "MMP", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20']].reset_index()
    score_dataframe_avg_df = b.replace({"resnet": "ResNet-101", "resnext": "ResNeXT-101"})
    
    return score_dataframe_avg_df

In [2]:
data_folder = "./../../../data/"
annotations_by_hand = "./../../../data/manual_annotation/annotated_drawing_artwork_pairs.csv"
uid2path = "./../../../data/text_data/uid2path.pkl"

drawings_embeddings_path_mini = "./../../../data/embeddings/drawings_learning_resnext-101_avg_280_ludovica_firsttrain.npy"
artworks_embeddings_path_mini = "./../../../data/embeddings/famous_artworks_resnext-101_avg_280_ludovica_firsttrain.npy"

drawings_embeddings_path_clus = "./../../../data/embeddings/drawings_learning_resnext-101_avg_280_ludovica_secondtrain.npy"
artworks_embeddings_path_clus = "./../../../data/embeddings/famous_artworks_resnext-101_avg_280_ludovica_secondtrain.npy"

In [3]:
manually_annotated_df = pd.read_csv(annotations_by_hand)
with open(uid2path, "rb") as outfile:
    uid2path_df = pickle.load(outfile)
    
drawings_embeddings_mini = np.load(drawings_embeddings_path_mini, allow_pickle=True)
artworks_embeddings_mini = np.load(artworks_embeddings_path_mini, allow_pickle=True)

drawings_embeddings_clus = np.load(drawings_embeddings_path_clus, allow_pickle=True)
artworks_embeddings_clus = np.load(artworks_embeddings_path_clus, allow_pickle=True)

number_of_artworks = len(artworks_embeddings_mini)

In [4]:
train_mets, val_mets,test_mets, non_train_mets = [], [], [], []

## Mini-Replica

In [5]:
draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split.csv")

draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

train_mets.append(("v0", "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["train"], number_of_artworks))

val_mets.append(("v0", "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["val"], number_of_artworks))

test_mets.append(("v0", "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["test"], number_of_artworks))

non_train_mets.append(("v0", "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 596/596 [01:02<00:00,  9.54it/s]


Mean position of artwork:  614.0982456140351
Mean of Minimum position of artwork:  736.3870967741935
Mean of median position of artwork:  590.1111111111111
Mean Average Precision:  0.20110348450862126
recall @ 400 0.6720430107526881
recall @ 200 0.5931899641577061
recall @ 100 0.5053763440860215
recall @ 50 0.4444444444444444
recall @ 20 0.35304659498207885


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 245.62it/s]


Mean position of artwork:  782.625
Mean of Minimum position of artwork:  1247.5263157894738
Mean of median position of artwork:  802.8157894736842
Mean Average Precision:  0.4279616877652773
recall @ 400 0.631578947368421
recall @ 200 0.631578947368421
recall @ 100 0.5789473684210527
recall @ 50 0.5263157894736842
recall @ 20 0.47368421052631576


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:02<00:00, 270.90it/s]


Mean position of artwork:  267.15
Mean of Minimum position of artwork:  118.73684210526316
Mean of median position of artwork:  275.0
Mean Average Precision:  0.4444008784033433
recall @ 400 0.7894736842105263
recall @ 200 0.7894736842105263
recall @ 100 0.631578947368421
recall @ 50 0.631578947368421
recall @ 20 0.631578947368421


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:04<00:00, 133.84it/s]


Mean position of artwork:  524.8875
Mean of Minimum position of artwork:  683.1315789473684
Mean of median position of artwork:  538.9078947368421
Mean Average Precision:  0.43618128308431037
recall @ 400 0.7105263157894737
recall @ 200 0.7105263157894737
recall @ 100 0.6052631578947368
recall @ 50 0.5789473684210527
recall @ 20 0.5526315789473685


In [6]:
for i in range(1, 11):
    
    lbl = "v"+str(i)
    
    draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split_"+lbl+".csv")
    
    draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

    train_mets.append((lbl, "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["train"], number_of_artworks))

    val_mets.append((lbl, "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["val"], number_of_artworks))

    test_mets.append((lbl, "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["test"], number_of_artworks))

    non_train_mets.append((lbl, "Mini-Replica") + get_scores(drawings_embeddings_mini, artworks_embeddings_mini, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 531/531 [00:44<00:00, 11.99it/s]


Mean position of artwork:  579.8511904761905
Mean of Minimum position of artwork:  660.6375
Mean of median position of artwork:  552.1208333333333
Mean Average Precision:  0.2416056887844027
recall @ 400 0.69375
recall @ 200 0.6229166666666667
recall @ 100 0.54375
recall @ 50 0.48333333333333334
recall @ 20 0.3958333333333333


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:02<00:00, 183.85it/s]


Mean position of artwork:  566.0
Mean of Minimum position of artwork:  732.0
Mean of median position of artwork:  566.0
Mean Average Precision:  0.24534886625298047
recall @ 400 0.7666666666666667
recall @ 200 0.5666666666666667
recall @ 100 0.4666666666666667
recall @ 50 0.3333333333333333
recall @ 20 0.26666666666666666


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:01<00:00, 276.00it/s]


...

Mean position of artwork:  558.5208333333334
Mean of Minimum position of artwork:  780.7727272727273
Mean of median position of artwork:  585.7954545454545
Mean Average Precision:  0.431117442949849
recall @ 400 0.7272727272727273
recall @ 200 0.7272727272727273
recall @ 100 0.5909090909090909
recall @ 50 0.5909090909090909
recall @ 20 0.5


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:01<00:00, 373.89it/s]


Mean position of artwork:  406.90625
Mean of Minimum position of artwork:  413.8125
Mean of median position of artwork:  406.90625
Mean Average Precision:  0.33506179733173913
recall @ 400 0.8125
recall @ 200 0.8125
recall @ 100 0.6875
recall @ 50 0.625
recall @ 20 0.5


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:03<00:00, 161.16it/s]


Mean position of artwork:  497.875
Mean of Minimum position of artwork:  626.2631578947369
Mean of median position of artwork:  510.4736842105263
Mean Average Precision:  0.3906729605843291
recall @ 400 0.7631578947368421
recall @ 200 0.7631578947368421
recall @ 100 0.631578947368421
recall @ 50 0.6052631578947368
recall @ 20 0.5


---

## Clus-Replica

In [7]:
draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split.csv")

draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

train_mets.append(("v0", "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["train"], number_of_artworks))

val_mets.append(("v0", "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["val"], number_of_artworks))

test_mets.append(("v0", "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["test"], number_of_artworks))

non_train_mets.append(("v0", "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 596/596 [00:55<00:00, 10.80it/s]


Mean position of artwork:  643.6157894736842
Mean of Minimum position of artwork:  768.4014336917563
Mean of median position of artwork:  592.516129032258
Mean Average Precision:  0.19924036392854028
recall @ 400 0.6666666666666666
recall @ 200 0.5716845878136201
recall @ 100 0.5
recall @ 50 0.4229390681003584
recall @ 20 0.3333333333333333


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:01<00:00, 315.35it/s]


Mean position of artwork:  664.0
Mean of Minimum position of artwork:  997.4736842105264
Mean of median position of artwork:  678.1052631578947
Mean Average Precision:  0.4925622224060261
recall @ 400 0.7368421052631579
recall @ 200 0.7368421052631579
recall @ 100 0.6842105263157895
recall @ 50 0.6842105263157895
recall @ 20 0.6842105263157895


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:01<00:00, 320.76it/s]


Mean position of artwork:  407.5
Mean of Minimum position of artwork:  251.26315789473685
Mean of median position of artwork:  345.5263157894737
Mean Average Precision:  0.33579641739146365
recall @ 400 0.7894736842105263
recall @ 200 0.6842105263157895
recall @ 100 0.631578947368421
recall @ 50 0.5789473684210527
recall @ 20 0.5263157894736842


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:03<00:00, 160.80it/s]


Mean position of artwork:  535.75
Mean of Minimum position of artwork:  624.3684210526316
Mean of median position of artwork:  511.8157894736842
Mean Average Precision:  0.4141793198987449
recall @ 400 0.7631578947368421
recall @ 200 0.7105263157894737
recall @ 100 0.6578947368421053
recall @ 50 0.631578947368421
recall @ 20 0.6052631578947368


In [8]:
for i in range(1, 11):
    
    lbl = "v"+str(i)
    
    draw_art_pairs_df_full = pd.read_csv("./../../../data/model_learning/drawing_artwork_pair_train_test_split_"+lbl+".csv")
    
    draw_art_pairs_df = draw_art_pairs_df_full[
        (
            (draw_art_pairs_df_full["set"].isin(["train"]))
            | (
                (draw_art_pairs_df_full["set"].isin(["test", "val"]))
                & (
                    draw_art_pairs_df_full["drawing_id"]
                    == draw_art_pairs_df_full["parent"]
                )
            )
        )
    ].reset_index(drop=True)

    train_mets.append((lbl, "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["train"], number_of_artworks))

    val_mets.append((lbl, "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["val"], number_of_artworks))

    test_mets.append((lbl, "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["test"], number_of_artworks))

    non_train_mets.append((lbl, "Clus-Replica") + get_scores(drawings_embeddings_clus, artworks_embeddings_clus, draw_art_pairs_df, ["val", "test"], number_of_artworks))

100%|████████████████████████████████████████████████████████████████████████████████| 531/531 [00:46<00:00, 11.32it/s]


Mean position of artwork:  681.9940476190476
Mean of Minimum position of artwork:  782.0375
Mean of median position of artwork:  609.2375
Mean Average Precision:  0.23522109073227035
recall @ 400 0.6833333333333333
recall @ 200 0.59375
recall @ 100 0.51875
recall @ 50 0.45625
recall @ 20 0.3958333333333333


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:02<00:00, 182.96it/s]


Mean position of artwork:  422.2
Mean of Minimum position of artwork:  444.4
Mean of median position of artwork:  422.2
Mean Average Precision:  0.23507131751684016
recall @ 400 0.7
recall @ 200 0.6333333333333333
recall @ 100 0.6
recall @ 50 0.5333333333333333
recall @ 20 0.4


100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [00:02<00:00, 262.79it/s]


...

Mean position of artwork:  606.9791666666666
Mean of Minimum position of artwork:  745.5
Mean of median position of artwork:  572.1136363636364
Mean Average Precision:  0.36911820847112886
recall @ 400 0.7727272727272727
recall @ 200 0.7272727272727273
recall @ 100 0.6363636363636364
recall @ 50 0.5454545454545454
recall @ 20 0.45454545454545453


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:01<00:00, 384.79it/s]


Mean position of artwork:  511.0
Mean of Minimum position of artwork:  622.0
Mean of median position of artwork:  511.0
Mean Average Precision:  0.4961191297235048
recall @ 400 0.75
recall @ 200 0.6875
recall @ 100 0.6875
recall @ 50 0.6875
recall @ 20 0.625


100%|███████████████████████████████████████████████████████████████████████████████| 596/596 [00:04<00:00, 142.38it/s]


Mean position of artwork:  568.5875
Mean of Minimum position of artwork:  693.5
Mean of median position of artwork:  546.3815789473684
Mean Average Precision:  0.4225922805773924
recall @ 400 0.7631578947368421
recall @ 200 0.7105263157894737
recall @ 100 0.6578947368421053
recall @ 50 0.6052631578947368
recall @ 20 0.5263157894736842


In [10]:
train_metrics_df = pd.DataFrame(train_mets, columns=["version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
val_metrics_df = pd.DataFrame(val_mets, columns=["version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
test_metrics_df = pd.DataFrame(test_mets, columns=["version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])
non_train_metrics_df = pd.DataFrame(non_train_mets, columns=["version", "model", "MP","MMP", "mean_median_position", 'MAP','R@400', 'R@200', 'R@100', 'R@50', 'R@20'])

In [11]:
train_metrics_df["MAP"] = 100 * train_metrics_df["MAP"]
train_metrics_df["R@400"] = 100 * train_metrics_df["R@400"]
train_metrics_df["R@200"] = 100 * train_metrics_df["R@200"]
train_metrics_df["R@100"] = 100 * train_metrics_df["R@100"]
train_metrics_df["R@50"] = 100 * train_metrics_df["R@50"]
train_metrics_df["R@20"] = 100 * train_metrics_df["R@20"]

val_metrics_df["MAP"] = 100 * val_metrics_df["MAP"]
val_metrics_df["R@400"] = 100 * val_metrics_df["R@400"]
val_metrics_df["R@200"] = 100 * val_metrics_df["R@200"]
val_metrics_df["R@100"] = 100 * val_metrics_df["R@100"]
val_metrics_df["R@50"] = 100 * val_metrics_df["R@50"]
val_metrics_df["R@20"] = 100 * val_metrics_df["R@20"]

test_metrics_df["MAP"] = 100 * test_metrics_df["MAP"]
test_metrics_df["R@400"] = 100 * test_metrics_df["R@400"]
test_metrics_df["R@200"] = 100 * test_metrics_df["R@200"]
test_metrics_df["R@100"] = 100 * test_metrics_df["R@100"]
test_metrics_df["R@50"] = 100 * test_metrics_df["R@50"]
test_metrics_df["R@20"] = 100 * test_metrics_df["R@20"]

non_train_metrics_df["MAP"] = 100 * non_train_metrics_df["MAP"]
non_train_metrics_df["R@400"] = 100 * non_train_metrics_df["R@400"]
non_train_metrics_df["R@200"] = 100 * non_train_metrics_df["R@200"]
non_train_metrics_df["R@100"] = 100 * non_train_metrics_df["R@100"]
non_train_metrics_df["R@50"] = 100 * non_train_metrics_df["R@50"]
non_train_metrics_df["R@20"] = 100 * non_train_metrics_df["R@20"]

In [23]:
train_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines_replica/train_scores_kfold.csv", index=False)
val_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines_replica/val_scores_kfold.csv", index=False)
test_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines_replica/test_scores_kfold.csv", index=False)
non_train_metrics_df.to_csv("./../../../data/model_learning/comparison/scores/baselines_replica/non_train_scores_kfold.csv", index=False)

In [12]:
train_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,Mini-Replica,614.098246,736.387097,590.111111,20.110348,67.204301,59.318996,50.537634,44.444444,35.304659
1,v1,Mini-Replica,579.851190,660.637500,552.120833,24.160569,69.375000,62.291667,54.375000,48.333333,39.583333
2,v2,Mini-Replica,568.450000,641.810976,543.108740,26.836561,71.747967,65.650407,59.146341,52.845528,43.089431
3,v3,Mini-Replica,504.789530,619.919913,507.133117,27.356674,74.242424,67.532468,61.688312,54.112554,46.320346
4,v4,Mini-Replica,531.323293,598.349794,519.090535,26.267754,73.251029,66.255144,58.641975,52.263374,44.032922
5,v5,Mini-Replica,656.153101,824.702811,634.287149,22.723035,65.461847,57.831325,49.598394,43.172691,35.943775
6,v6,Mini-Replica,558.261179,679.052083,541.163542,25.163586,70.208333,63.750000,56.458333,50.833333,42.500000
7,v7,Mini-Replica,494.591228,554.211470,478.514337,22.731284,71.326165,64.157706,55.197133,48.745520,39.247312
8,v8,Mini-Replica,575.304264,726.606426,562.259036,26.290444,69.678715,63.052209,55.421687,49.598394,41.767068
9,v9,Mini-Replica,638.572500,799.484375,617.910590,19.031073,65.625000,58.159722,49.305556,42.534722,35.069444


In [13]:
val_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,Mini-Replica,782.625000,1247.526316,802.815789,42.796169,63.157895,63.157895,57.894737,52.631579,47.368421
1,v1,Mini-Replica,566.000000,732.000000,566.000000,24.534887,76.666667,56.666667,46.666667,33.333333,26.666667
2,v2,Mini-Replica,564.954545,729.909091,564.954545,28.313360,63.636364,59.090909,50.000000,50.000000,50.000000
3,v3,Mini-Replica,445.050000,425.689655,421.741379,13.911420,72.413793,51.724138,41.379310,31.034483,24.137931
4,v4,Mini-Replica,533.145161,674.166667,546.983333,10.060917,70.000000,50.000000,36.666667,26.666667,20.000000
5,v5,Mini-Replica,343.431034,286.862069,343.431034,31.862723,79.310345,58.620690,51.724138,41.379310,37.931034
6,v6,Mini-Replica,459.275000,408.277778,441.250000,34.946418,72.222222,72.222222,72.222222,66.666667,50.000000
7,v7,Mini-Replica,835.500000,1271.000000,835.500000,28.069821,68.421053,63.157895,52.631579,52.631579,47.368421
8,v8,Mini-Replica,538.451613,585.466667,528.266667,16.062069,63.333333,43.333333,40.000000,30.000000,26.666667
9,v9,Mini-Replica,433.500000,467.000000,433.500000,42.331568,78.947368,73.684211,57.894737,52.631579,52.631579


In [14]:
test_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,Mini-Replica,267.150000,118.736842,275.000000,44.440088,78.947368,78.947368,63.157895,63.157895,63.157895
1,v1,Mini-Replica,419.071429,438.142857,419.071429,30.037969,80.952381,80.952381,76.190476,76.190476,66.666667
2,v2,Mini-Replica,473.553571,551.444444,486.722222,12.085234,74.074074,51.851852,40.740741,25.925926,18.518519
3,v3,Mini-Replica,579.222222,665.120000,587.080000,23.532902,60.000000,56.000000,44.000000,44.000000,36.000000
4,v4,Mini-Replica,459.261905,426.550000,426.175000,36.351535,65.000000,65.000000,55.000000,55.000000,40.000000
5,v5,Mini-Replica,269.925000,124.578947,277.921053,23.759488,84.210526,84.210526,73.684211,73.684211,63.157895
6,v6,Mini-Replica,430.727273,461.454545,430.727273,27.557347,75.757576,54.545455,48.484848,39.393939,36.363636
7,v7,Mini-Replica,720.547619,991.631579,730.973684,42.813843,63.157895,52.631579,52.631579,52.631579,47.368421
8,v8,Mini-Replica,337.555556,275.111111,337.555556,35.985550,83.333333,83.333333,77.777778,72.222222,66.666667
9,v9,Mini-Replica,259.656250,119.312500,259.656250,49.288463,87.500000,75.000000,75.000000,75.000000,62.500000


In [15]:
non_train_metrics_df

,version,model,MP,MMP,mean_median_position,MAP,R@400,R@200,R@100,R@50,R@20
0,v0,Mini-Replica,524.887500,683.131579,538.907895,43.618128,71.052632,71.052632,60.526316,57.894737,55.263158
1,v1,Mini-Replica,505.500000,611.000000,505.500000,26.800862,78.431373,66.666667,58.823529,50.980392,43.137255
2,v2,Mini-Replica,513.770000,631.571429,521.846939,19.371331,69.387755,55.102041,44.897959,36.734694,32.653061
3,v3,Mini-Replica,508.605263,536.537037,498.287037,18.365810,66.666667,53.703704,42.592593,37.037037,29.629630
4,v4,Mini-Replica,503.307692,575.120000,498.660000,20.577164,68.000000,56.000000,44.000000,38.000000,28.000000
5,v5,Mini-Replica,313.428571,222.625000,317.500000,28.655193,81.250000,68.750000,60.416667,54.166667,47.916667
6,v6,Mini-Replica,441.500000,442.686275,434.441176,30.165254,74.509804,60.784314,56.862745,49.019608,41.176471
7,v7,Mini-Replica,775.150000,1131.315789,783.236842,35.441832,65.789474,57.894737,52.631579,52.631579,47.368421
8,v8,Mini-Replica,464.653061,469.083333,456.750000,23.533374,70.833333,58.333333,54.166667,45.833333,41.666667
9,v9,Mini-Replica,354.028571,308.057143,354.028571,45.511863,82.857143,74.285714,65.714286,62.857143,57.142857


In [3]:
train_metrics_df= pd.read_csv("./../../../data/model_learning/comparison/scores/baselines_replica/train_scores_kfold.csv")
val_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines_replica/val_scores_kfold.csv")
test_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines_replica/test_scores_kfold.csv")
non_train_metrics_df = pd.read_csv("./../../../data/model_learning/comparison/scores/baselines_replica/non_train_scores_kfold.csv")

In [4]:
get_averaged_df(train_metrics_df)

C:\Users\ravin\AppData\Local\Temp\ipykernel_26512\3749065824.py:2: FutureWarning: ['version'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  a = score_dataframe.groupby("model").agg(['mean', 'std'])


,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,Clus-Replica,637.17 ± 49.67,769.36 ± 72.87,23.15 ± 2.47,69.06 ± 1.99,59.73 ± 2.33,53.38 ± 2.5,46.76 ± 3.19,39.17 ± 3.6
1,Mini-Replica,575.88 ± 52.21,688.71 ± 84.05,23.8 ± 2.87,69.56 ± 2.97,62.43 ± 3.44,54.69 ± 4.15,48.34 ± 4.06,39.9 ± 3.93


In [5]:
get_averaged_df(val_metrics_df)

C:\Users\ravin\AppData\Local\Temp\ipykernel_26512\3749065824.py:2: FutureWarning: ['version'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  a = score_dataframe.groupby("model").agg(['mean', 'std'])


,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,Clus-Replica,587.53 ± 128.07,701.14 ± 257.2,27.35 ± 9.6,68.86 ± 6.24,61.6 ± 7.49,56.22 ± 7.23,50.81 ± 8.6,41.11 ± 11.4
1,Mini-Replica,550.95 ± 145.63,691.7 ± 321.34,28.73 ± 11.77,70.99 ± 5.94,60.4 ± 9.88,51.47 ± 10.25,45.1 ± 13.32,39.34 ± 12.55


In [6]:
get_averaged_df(test_metrics_df)

C:\Users\ravin\AppData\Local\Temp\ipykernel_26512\3749065824.py:2: FutureWarning: ['version'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  a = score_dataframe.groupby("model").agg(['mean', 'std'])


,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,Clus-Replica,526.46 ± 139.92,536.8 ± 214.68,33.09 ± 8.47,76.37 ± 8.65,67.11 ± 5.78,63.04 ± 6.7,58.1 ± 8.99,49.34 ± 10.43
1,Mini-Replica,420.33 ± 140.99,416.9 ± 263.74,32.67 ± 10.78,75.83 ± 9.27,69.43 ± 13.49,61.4 ± 13.76,58.16 ± 16.5,50.04 ± 15.93


In [7]:
get_averaged_df(non_train_metrics_df)

C:\Users\ravin\AppData\Local\Temp\ipykernel_26512\3749065824.py:2: FutureWarning: ['version'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  a = score_dataframe.groupby("model").agg(['mean', 'std'])


,model,MP,MMP,MAP,R@400,R@200,R@100,R@50,R@20
0,Clus-Replica,555.1 ± 103.66,617.31 ± 163.74,29.61 ± 7.77,72.2 ± 5.27,64.21 ± 5.06,59.12 ± 5.21,53.57 ± 6.6,44.23 ± 8.74
1,Mini-Replica,491.16 ± 116.86,567.04 ± 235.21,30.1 ± 9.64,73.19 ± 5.88,63.54 ± 8.15,54.89 ± 8.02,49.61 ± 9.32,43.09 ± 9.79
